# Spam Dection using NLP

By: Kritarth Shah

For this program, I have only used the first 4000 messages of the file since my system was unable to allocate an array with that large of a size. If you system can handle the space needed, feel free to remove the "[:4000]" line below.

## Step 1: Reading Data

In [1]:
import pandas as pd

data = pd.read_csv("spam.csv", encoding="latin-1")
data = data[['v1', 'v2']][:4000]
data.columns = ['label', 'text']

data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Step 2: Cleaning Data

For the ease of understanding this project, the following methods are divided into their own functions and later called in one function

### Sub 1: Removing Punctuations

In [2]:
import string

def punctuation_remove(text):
    no_punct = "".join([char for char in text if char not in string.punctuation])
    return(no_punct.lower())

### Sub 2: Tokenizing Words  

In [3]:
import re

def tokenizer(text):
    tokens = re.split('\W+', text)
    return tokens

### Sub 3: Removing Stopwords

In [4]:
import nltk

stopwords = nltk.corpus.stopwords.words('english')

def stopwords_remove(text):
    no_stopwords = [word for word in text if word not in stopwords]
    return no_stopwords

### Sub 4: Lemmatizing Words

In [5]:
lemwords = nltk.WordNetLemmatizer()

def lemmatizer(text):
    lemmatized = [lemwords.lemmatize(word) for word in text]
    return lemmatized

### Cleaning Data (with previous four functions)

In [6]:
def clean_data(text):
    return lemmatizer(stopwords_remove(tokenizer(punctuation_remove(text))))

data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Step 3: Adding Features

In [7]:
#add data of length of every mail minus white spaces
data['body_len'] = data['text'].apply(lambda x: len(x) - x.count(" "))

#function to get % of punctuations in the whole text minus white space
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

#add data of % of punctuations in text
data['punct%'] = data['text'].apply(lambda x: count_punct(x))

## Step 3: Vectorizing Data

### Sub 1: Spliting Data into test and train

In [8]:
from sklearn.model_selection import train_test_split

#splitting the test and training data (using data of 5 (20%))
X_train, X_test, y_train, y_test = train_test_split(data[['text', 'body_len', 'punct%']], data['label'], test_size=0.2)

### Sub 2: Vectorizing test and training data

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer=clean_data)
tfidf_vect_fit = tfidf_vect.fit(X_train['text'])

tfidf_train = tfidf_vect_fit.transform(X_train['text'])
X_train_vect = pd.concat([X_train[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray())], axis=1)

tfidf_test = tfidf_vect_fit.transform(X_test['text'])
X_test_vect = pd.concat([X_test[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray())], axis=1)

In [10]:
print("Training Data Vector")
X_train_vect.head()

Training Data Vector


,body_len,punct%,0,1,2,3,4,5,6,7,...,6483,6484,6485,6486,6487,6488,6489,6490,6491,6492
0,42,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,41,7.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,33,6.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,57,8.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
print("Testing Data Vector")
X_test_vect.head()

Testing Data Vector


,body_len,punct%,0,1,2,3,4,5,6,7,...,6483,6484,6485,6486,6487,6488,6489,6490,6491,6492
0,16,18.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,131,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,48,16.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# print(tfidf_train.shape)
# print(tfidf_test.shape)
# print(tfidf_vect.get_feature_names())

(3200, 6493)
(800, 6493)
['', '008704050406', '01223585236', '01223585334', '0125698789', '020603', '0207', '02072069400', '02085076972', '021', '050703', '0578', '06', '07046744435', '07090201529', '07090298926', '071104', '07123456789', '0721072', '07732584351', '07734396839', '07742676969', '07753741225', '07786200117', '077xxx', '07801543489', '07808', '07815296484', '07821230901', '0794674629107880867867', '07xxxxxxxxx', '0800', '08000407165', '08000776320', '08000839402', '08000930705', '08000938767', '08001950382', '08002888812', '08002986030', '08002986906', '08002988890', '08006344447', '0808', '08081263000', '08081560665', '0825', '0844', '08448714184', '0845', '08452810071', '08452810073', '08452810075over18s', '0870', '08700621170150p', '08701237397', '08701417012', '08701417012150p', '0870141701216', '087016248', '08701752560', '087018728737', '0870241182716', '08702490080', '08702840625', '08702840625comuk', '08704439680', '08706091795', '08707509020', '0870753331018', '0

## Step 4: Creating ML Classifier

In [13]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix

###### Note
The parameters use for the classifier below were determined to be the most efficient and effetive from a combination of variables that were tested. To find these tests, check out the classifier parameter file.

In [14]:
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

rf_model = rf.fit(X_train_vect, y_train) #fitting training data with label
y_pred = rf_model.predict(X_test_vect)

In [15]:
precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')

matrix = confusion_matrix(y_test, y_pred)

pHam_Ham = matrix[0][0]
pHam_Spam = matrix[1][0]
pSpam_Ham = matrix[0][1]
pSpam_Spam = matrix[1][1]

print("------------Results-----------")
print("Predicted to be Ham and is Ham ->", pHam_Ham, "  ---- GOOD")
print("Predicted to be Ham and is Spam ->", pHam_Spam, "  ---- False Negative")
print("Predicted to be Spam and is Ham ->", pSpam_Ham, "  ---- False Positive")
print("Predicted to be Spam and is Spam ->", pSpam_Spam, "  ---- GOOD")
print("------------Stats-------------")
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))


------------Results-----------
Predicted to be Ham and is Ham -> 695   ---- GOOD
Predicted to be Ham and is Spam -> 17   ---- False Negative
Predicted to be Spam and is Ham -> 0   ---- False Positive
Predicted to be Spam and is Spam -> 88   ---- GOOD
------------Stats-------------
Precision: 1.0 / Recall: 0.838 / Accuracy: 0.979


Precision -> The number of mails that were predicted to be spam that are actually spam

Recall -> The amount of mails that were perdicted to be spam divided by the total amount of spam

Accuracy -> The number of correct mail labels that were given to all test cases

## End Notes

As seen from the results, from using the Random Forest Classifier, we are able to predict the correct ham/spam option with a 98% accuracy with a few  false positives. For this program, the Gradient Boosting Classifier was not used because if you look at the Classifier notes, Gradient does not have a 100% percision. In the case of spam/ham, having false positives if ok, meaning there will be some spam in your ham mail; but having important mails in spam is not good. So with the given situation, the Randon Forest Classifier was the better options. To further imporve the results, in the future we can add more features and try more variales for the classifier parameters.